## Fundus imageset.
Code to download the fundus retina images from kaggle.

In [3]:
import kagglehub

path = kagglehub.dataset_download("linchundan/fundusimage1000")

print("Path to dataset files:", path)


100%|██████████| 778M/778M [00:38<00:00, 21.0MB/s] 

Extracting files...


Path to dataset files: /Users/benmurarotto/.cache/kagglehub/datasets/linchundan/fundusimage1000/versions/4


## Data Loader for OCTDataset
from reference code

In [4]:
import os
import numpy as np

import cv2
import nibabel as nib
from skimage import transform

print(os.getcwd())

PATH_TO_DATA = "../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]" 

def read_data(image_size=(50, 50, 1), address='./images', flip=False, debug=False):
    folder_list = [item[0] for item in os.walk(address)]
    print(folder_list)
    folder_list = folder_list[1:]
    
    if address[-4:] == '.npz':
        data = np.load(address)
        images = np.vstack((data['train_images'], data['val_images'], data['test_images']))
        labels = np.vstack((data['train_labels'], data['val_labels'], data['test_labels']))
        octs = []
        oct_label = []
        for i, d in enumerate(images):
            d_ = d.astype('float')
            d = d.astype('float')
            if len(d_.shape) == 2:
                d_ = np.tile(np.expand_dims(d_, 2), 3)
                d = np.tile(np.expand_dims(d, 2), 3)
            d_[:, :, 2] = d[:, :, 0]
            d_[:, :, 0] = d[:, :, 2]
            d_ /= np.max(d_)
            oct_image_standard_size = transform.resize(d_, image_size, order=1, preserve_range=True)
            octs.append(oct_image_standard_size)
            if labels[i] == 0:
                oct_label.append(0)
            else:
                oct_label.append(1)
        print("Fraction of AD images:", sum(oct_label)/len(oct_label))
    
        return np.asarray(octs), np.asarray(oct_label), address
    
    octs = []
    oct_label = []
    addresses = []
    label_counter = 0
    for _folder in folder_list:
        file_list = os.listdir(os.path.join(os.getcwd(), _folder))
        for _file in file_list:
            #if (_file[-4:] == '.tif') or (_file[-4:] == '.JPG'):
            if (_file[-4:] == '.png') or (_file[-4:] == '.JPG') or (_file[-5:] == '.jpeg') or (_file[-4:] == '.img'): ## aded this extra part for img files
                oct_image = cv2.imread(os.path.join(os.getcwd(), _folder, _file)).astype('float64')
                addresses.append(os.path.join(os.getcwd(), _folder, _file))
                oct_image /= np.max(oct_image)
                
                oct_image_standard_size = np.zeros(image_size)
                larger_side = np.argmax(oct_image.shape)
                smaller_side = 1-larger_side
                scaling_factor = image_size[0] / oct_image.shape[larger_side]
                if larger_side == 0:
                    image_ = transform.resize(oct_image, (image_size[0], int(scaling_factor*oct_image.shape[smaller_side]), image_size[2]), order=1, preserve_range=True)
                else:
                    image_ = transform.resize(oct_image, (int(scaling_factor*oct_image.shape[smaller_side]), image_size[1], image_size[2]), order=1, preserve_range=True)
                for i in range(image_.shape[0]):
                    for j in range(image_.shape[1]):
                        oct_image_standard_size[i, j, :] = image_[i, j, :]
                if address.find('OCTID') != -1:
                    oct_image_standard_size = transform.rotate(oct_image_standard_size, 270)
                
                octs.append(oct_image_standard_size)
                if flip:
                    oct_image_flipped = cv2.flip(oct_image_standard_size, 1).reshape(image_size)
                    octs.append(oct_image_flipped)
                
                if _folder.find('hc') != -1:
                    oct_label.append(0)
                    if flip:
                        oct_label.append(0)
                else:
                    oct_label.append(1)
                    if flip:
                        oct_label.append(1)
                
                if debug:
                    print(_file)
                
                
    label_counter = 2
    
    print("Fraction of AD images:", sum(oct_label)/len(oct_label))
    
    return np.asarray(octs), np.asarray(oct_label), addresses
    
if __name__ == '__main__':
    x, y, _ = read_data(image_size=(224, 224, 3), address=PATH_TO_DATA, debug=False)
    for i in range(x.shape[0]):
        if y[i] == 0:
            cv2.imwrite('./oct_data/hc/data_'+str(i)+'.png', 255*x[i])
        else:
            cv2.imwrite('./oct_data/ad/data_'+str(i)+'.png', 255*x[i])


/media/benmurarotto/BensFiles/Dev/GitRepos/deep-learning-alzheimers/notebooks
['../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/CO100', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD057', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD362', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD387', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD414', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD439', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD443', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD459', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD508', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD613', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD620', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD647', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD727', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD786', '../data/[[[OCT RAW FOR DRYAD]]][2020-01JAN-16]/AD807', '../data/[[[OCT RAW FOR DRYAD]]

AttributeError: 'NoneType' object has no attribute 'astype'

In [2]:
print(x.shape)

(1, 224, 224, 3)


In [3]:
import tensorflow as tf
print("GPU Available: ", tf.config.list_physical_devices('GPU')) ### checking GPU / CUDA compat

2025-08-01 10:44:09.491517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754009049.511800    9494 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754009049.516863    9494 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
total_num_images=len(image_paths)
epochs = 20
batch_size = 2 